Evaluating Performance of Zero-Shot Inference

In [1]:
import re

import nltk
import pandas as pd
nltk.download('punkt_tab')

from evaluate import load
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import corpus_bleu

from rouge_score import rouge_scorer
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/work5/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/opt/miniconda3/envs/project_news_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to /home/work5/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
df_path = '{your_dataset_path}'

df = pd.read_csv(df_path)

df.drop(columns = ['Unnamed: 0'], inplace = True)

df.head(1)

,Заголовок,Текст новости,Дата публикации,Тэги,prompt,unsloth/gemma-2-2b_headline,unsloth/gemma-2-9b-bnb-4bit_headline,unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit_headline
0,Банки предложили блокировать карты россиян при...,Банки выступили с инициативой блокировать карт...,2019-08-20,"'Томск', 'Экономика', 'Банки'",Ты - профессиональный журналист с многолетним ...,Ответ:\nБанки выступили с инициативой блокиров...,Ответ:\nБанки выступили с инициативой блокиров...,Ответ:\nБанки предложили блокировать карты кли...


I. BertScore

In [6]:
def bert_score(refs: list, preds: list)->dict:
    '''Функция расчитывает средний показатель метрики BertScore.


    Args:
        refs (list): Список образцов-эталонов.
        preds (list): Список образцов-кандидатов.
    
        
    Returns:
        dict - словарь, содержащий в себе следующие ключи:
            Mean Precision - Средний показатель метрики BERT-P
            Mean Recall - Средний показатель метрики BERT-R
            Mean F1 - Средне-гармоническое меджду BERT-P и BERT-R

    '''
    bert_score = load('bertscore')

    predictions = preds

    references = refs

    result = bert_score.compute(predictions = predictions, references = references, lang = 'ru')

    mean_precision = sum(result['precision']) / len(result['precision'])

    mean_recall = sum(result['recall']) / len(result['recall'])

    mean_f1 = sum(result['f1']) / len(result['f1'])

    united_dictionary = {
        'Mean Precision': round(mean_precision, 2),
        'Mean Recall': round(mean_recall, 2),
        'Mean F1': round(mean_f1, 2)
    }

    return united_dictionary

Оценка результатов, сгенерированных моделями.

In [5]:
col_to_evaluate = list(df.columns[5:]) #Переменная

model_name_extractions_pattern = r".*[^_headline]"

reference_col = df['Заголовок'].tolist()

for col in col_to_evaluate:
    model_name = re.findall(model_name_extractions_pattern, col)[0]
    
    print(f"Показатели метрики BertScore для модели {model_name}: ")

    results = bert_score(refs = reference_col, preds = df[col].tolist())

    print(results)

Показатели метрики BertScore для модели unsloth/gemma-2-2b: 
{'Mean Precision': 0.61, 'Mean Recall': 0.68, 'Mean F1': 0.64}
Показатели метрики BertScore для модели unsloth/gemma-2-9b-bnb-4bit: 
{'Mean Precision': 0.7, 'Mean Recall': 0.75, 'Mean F1': 0.72}
Показатели метрики BertScore для модели unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit: 
{'Mean Precision': 0.72, 'Mean Recall': 0.76, 'Mean F1': 0.74}


В целом, модели Gemma2:9b и Mistral_Nemo демонстрируют приемлемые результаты семантического, даже при Zero-Shot Inference

II. BLEU Score

In [6]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import corpus_bleu
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/work5/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
def calculate_bleu_score(refs:str, preds:str)->float:
    '''Функция выполняет расчет показателя метрики BLEU по двум объектам.
    Args:
        refs (str): Образцы-эталоны
        preds (str): Образцы-кандидаты (предсказания модели)
    Returns:
        float - числовой показатель метрики BLEU
    '''
    smooth = SmoothingFunction() #Важно добавить SmoothingFunction, поскольку если нет пересекающихся n-грамм, BLEU будет равен 0

    refs_tokenized, preds_tokenized = word_tokenize(refs), word_tokenize(preds)

    weights = (0.25, 0.25, 0.25, 0.25)

    return sentence_bleu(references = refs_tokenized, hypothesis = preds_tokenized,
                         weights = weights, smoothing_function=smooth.method4)

Функция расчитывающая Bleu_score на всем корпусе

In [49]:
def calculate_bleu_corpus(references: list, predictions: list) -> float:
    '''Функция выполняет расчет метрики BLEU по всему корпусу.

    Args:

        references (list): Список, содержащий в себе образцы-эталоны.
        predictions (list): Список, содержащий в себе образцы-кандидаты, т.е. сгенерированные моделью образцы
    
    Returns:
        float - числовой показатель метрики BLEU
    '''

    for index in range(0, len(references)):
        references[index] = word_tokenize(references[index], language = 'russian')
    
    for index in range(0, len(predictions)):
        predictions[index] = word_tokenize(predictions[index], language = 'russian')
    
    weights = (0.25, 0.25, 0.25, 0.25)

    smooth = SmoothingFunction()

    return round(corpus_bleu(
        list_of_references = references,
        hypotheses = predictions,
        weights = weights,
        smoothing_function = smooth.method4
    ), 5)

In [9]:
#Расчет BLEU-Score
for col in col_to_evaluate:
    model_name = re.findall(model_name_extractions_pattern, col)[0]

    reference_col = df['Заголовок'].tolist()

    print(model_name)
    
    print(f"Показатели метрики BertScore для модели {model_name}: ")

    results = calculate_bleu_corpus(references = reference_col, predictions = df[col].tolist())

    print(results)

unsloth/gemma-2-2b
Показатели метрики BertScore для модели unsloth/gemma-2-2b: 
0.00029
unsloth/gemma-2-9b-bnb-4bit
Показатели метрики BertScore для модели unsloth/gemma-2-9b-bnb-4bit: 
0.00038
unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit
Показатели метрики BertScore для модели unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit: 
0.00074


ROUGE-Score calculation

In [3]:
def calculate_rouge_score(prediction: str, reference: str)-> str:
    '''Функция расчитывает показатели метрики Rouge-1, Rouge-2 и Rouge-L
    Args:
        predictions (str): Строка, содержщая предсказание модели
        reference (str): Строка, содержащая эталонную строку
    Returns:
        dict - словарь, содержащий в себе показатели F-меры метрик, указанных выше 
    '''
    
    def tokenize_russian(text):
        '''Мы должны добавить дополнительную функцию токенизации на русском языке,
        посколько функции расчета метрики Rouge из библиотеки rouge_score могут обрабатывать русские предложения некорректно'''
        return ' '.join(word_tokenize(text, language = 'russian'))
    
    prediction = tokenize_russian(prediction)
    reference = tokenize_russian(reference)

    class SimpleTokenizer:
        def tokenize(self, text):
            return text.split()
    
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'],
                                      tokenizer = SimpleTokenizer())
    
    results = scorer.score(prediction, reference)

    final_dict = {
        'Rouge1 F-measure': results['rouge1'].fmeasure,
        'Rouge2 F-measure': results['rouge2'].fmeasure,
        'RougeL F-measure': results['rougeL'].fmeasure,
    }

    return final_dict

In [12]:
for col in col_to_evaluate:

    df[f'rouge_score_of_{col}'] = '-'

reference_col = 'Заголовок'

for col in col_to_evaluate:
    
    current_column = col #Содержит текущую колонку

    for index, row in df.iterrows():
        
        #current_index = index #Содержит индекс текущей итерации

        current_headline = df.at[index, reference_col] #Содержит образцовый заголовок

        current_prediction = df.at[index, current_column] #Содержит предсказание

        df.at[index, f'rouge_score_of_{col}'] = calculate_rouge_score(prediction = current_prediction,
                                                            reference = current_headline)


In [72]:
def calculate_mean_rouge_score(df_column: str)-> dict:
    '''Функция расчитывает средние показатели метрик Rouge-1, Rouge-2, Rouge-L.
    
    Args:
        df_column (str): Название колонки в датафрейме, содержащая показатели приведенных выше метрик в виде словаря.

    Returns:
        dict - словарь, содержащий в себе средние показатели приведенных выше метрик.
    '''

    rouge_1 = []

    rouge_2 = []

    rouge_L = []

    for index, row in df.iterrows():
        rouge_1.append(row[df_column]['Rouge1 F-measure'])
        rouge_2.append(row[df_column]['Rouge2 F-measure'])
        rouge_L.append(row[df_column]['RougeL F-measure'])
    
    final_dict = {
        'Mean Rouge1 Score': round(sum(rouge_1) / len(rouge_1), 2),
        'Mean Rouge2 Score': round(sum(rouge_2) / len(rouge_2), 2),
        'Mean RougeL Score': round(sum(rouge_L) / len(rouge_L), 2),
    }
    
    return final_dict

In [17]:
for col in col_to_evaluate:
    print(f"Показатели метрики Rouge для модели {re.findall(model_name_extractions_pattern, col)[0]} Следующие: ")

    print(calculate_mean_rouge_score(f"rouge_score_of_{col}"))

Показатели метрики Rouge для модели unsloth/gemma-2-2b Следующие: 
{'Mean Rouge1 Score': 0.1, 'Mean Rouge2 Score': 0.03, 'Mean RougeL Score': 0.08}
Показатели метрики Rouge для модели unsloth/gemma-2-9b-bnb-4bit Следующие: 
{'Mean Rouge1 Score': 0.22, 'Mean Rouge2 Score': 0.11, 'Mean RougeL Score': 0.2}
Показатели метрики Rouge для модели unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit Следующие: 
{'Mean Rouge1 Score': 0.24, 'Mean Rouge2 Score': 0.11, 'Mean RougeL Score': 0.22}


Fine-Tuned Models Evaluation

In [2]:
finetuned_df = pd.read_csv('{your_finetuned_dataset_path}')
finetuned_df.drop(columns = ['Unnamed: 0'], inplace = True)
finetuned_df.rename(columns = {'unsloth/Mistral-Nemo_Finetuned_generated_headline':'unsloth/mistral-nemo_finetuned_generated_headline'}, inplace = True)

I. BertScore Calculation

In [7]:
all_cols = list(finetuned_df.columns)

needed_cols = []

model_extraction_pattern = r"unsloth/(.*?)_finetuned_generated_headline"

for col in all_cols:
    if col.startswith('unsloth'):
        needed_cols.append(col)

reference_col = finetuned_df['Заголовок'].tolist()

for column in needed_cols:
    preds = finetuned_df[column].tolist()

    print(f"Для модели {re.findall(model_extraction_pattern, column)[0]} показатели метрики BertScore следующие: ")

    print(bert_score(refs = reference_col, preds = preds))

Для модели gemma2_2b показатели метрики BertScore следующие: 
{'Mean Precision': 0.77, 'Mean Recall': 0.77, 'Mean F1': 0.77}
Для модели mistral-nemo показатели метрики BertScore следующие: 
{'Mean Precision': 0.79, 'Mean Recall': 0.79, 'Mean F1': 0.79}
Для модели gemma-2_9b показатели метрики BertScore следующие: 
{'Mean Precision': 0.79, 'Mean Recall': 0.79, 'Mean F1': 0.79}


II. Calculating BLEU Score

In [65]:
for column in needed_cols:
    reference_col = finetuned_df['Заголовок'].tolist()

    print(f"Для модели {re.findall(model_extraction_pattern, column)[0]} показатель BLEU-Score составил: ")

    print(calculate_bleu_corpus(references = reference_col, predictions = finetuned_df[column].tolist()))

Для модели gemma2_2b показатель BLEU-Score составил: 
0.00079
Для модели mistral-nemo показатель BLEU-Score составил: 
0.00082
Для модели gemma-2_9b показатель BLEU-Score составил: 
0.0008


III. Rouge-Score Calculation

In [8]:
for col in needed_cols:

    finetuned_df[f'rouge_score_of_{col}'] = '-'

reference_col = 'Заголовок'

for col in needed_cols:
    
    current_column = col #Содержит текущую колонку

    for index, row in finetuned_df.iterrows():
        
        #current_index = index #Содержит индекс текущей итерации

        current_headline = finetuned_df.at[index, reference_col] #Содержит образцовый заголовок

        current_prediction = finetuned_df.at[index, current_column] #Содержит предсказание

        finetuned_df.at[index, f'rouge_score_of_{col}'] = calculate_rouge_score(prediction = current_prediction,
                                                            reference = current_headline)


In [9]:
#Немного изменим наш изначальный концепт функции расчета среднего Rouge
def calculate_mean_rouge_score(df: pd.DataFrame, df_column: str)-> dict:
    '''Функция расчитывает средние показатели метрик Rouge-1, Rouge-2, Rouge-L.
    
    Args:
        df_column (str): Название колонки в датафрейме, содержащая показатели приведенных выше метрик в виде словаря.

    Returns:
        dict - словарь, содержащий в себе средние показатели приведенных выше метрик.
    '''

    rouge_1 = []

    rouge_2 = []

    rouge_L = []

    for index, row in df.iterrows():
        rouge_1.append(row[df_column]['Rouge1 F-measure'])
        rouge_2.append(row[df_column]['Rouge2 F-measure'])
        rouge_L.append(row[df_column]['RougeL F-measure'])
    
    final_dict = {
        'Mean Rouge1 Score': round(sum(rouge_1) / len(rouge_1), 2),
        'Mean Rouge2 Score': round(sum(rouge_2) / len(rouge_2), 2),
        'Mean RougeL Score': round(sum(rouge_L) / len(rouge_L), 2),
    }
    
    return final_dict

In [10]:
for col in needed_cols:
    print(f"Показатели метрики Rouge для модели {re.findall(model_extraction_pattern, col)[0]} Следующие: ")

    #print(col)

    print(calculate_mean_rouge_score(df = finetuned_df, df_column = f"rouge_score_of_{col}"))

Показатели метрики Rouge для модели gemma2_2b Следующие: 
{'Mean Rouge1 Score': 0.31, 'Mean Rouge2 Score': 0.16, 'Mean RougeL Score': 0.29}
Показатели метрики Rouge для модели mistral-nemo Следующие: 
{'Mean Rouge1 Score': 0.34, 'Mean Rouge2 Score': 0.19, 'Mean RougeL Score': 0.32}
Показатели метрики Rouge для модели gemma-2_9b Следующие: 
{'Mean Rouge1 Score': 0.36, 'Mean Rouge2 Score': 0.21, 'Mean RougeL Score': 0.34}
